In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import cv2

# Define neural network architecture (CNN)
def create_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes)
    ])
    return model

In [20]:
# Define environment
class ImageEnvironment:
    def __init__(self, dataset):
        self.dataset = dataset
        self.num_samples = len(dataset)
        self.current_index = 0

    def reset(self):
        self.current_index = 0

    def step(self, action):
        image, true_label = self.dataset[self.current_index]
        reward = 1 if action == true_label else -1
        self.current_index = (self.current_index + 1) % self.num_samples
        return image, reward

In [21]:
def train_model(model, environment, optimizer, num_episodes):
    for episode in range(num_episodes):
        state, _ = environment.step(np.random.randint(0, num_classes))  # Initial state (random action)
        state = np.expand_dims(state, axis=0)  # Add batch dimension
        with tf.GradientTape() as tape:
            q_values = model(state, training=True)
            action = tf.argmax(q_values, axis=1)[0].numpy()
            _, reward = environment.step(action)
            target = reward + tf.reduce_max(q_values)
            loss = tf.square(target - q_values[0, action])
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        print(f"Episode: {episode+1}, Loss: {loss.numpy()}, Reward: {reward}")
    model.save(os.path.join('C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Models','rein_model.keras'))

In [22]:
# Example dataset (replace with your dataset)
def load_image(filepath):
    # Load image using OpenCV
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    # Resize image to 28x28
    image = cv2.resize(image, (28, 28))
    # Normalize pixel values
    image = image / 255.0
    return image

In [23]:
file_path = "C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original"
labels = ['Happy', 'Sad', 'Fear', 'Angry', 'Neutral']
dataset = []

for index, label in enumerate(labels):
    for filename in os.listdir(os.path.join(file_path,label)):
        image_path = os.path.join(file_path,label,filename)
        print(image_path)
        dataset.append((load_image(image_path),index))

C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-1.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-10.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-11.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-12.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-13.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-14.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-15.jpeg


C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-16.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-17.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-18.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-19.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-2.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-20.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-21.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-22.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-23.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Datasets\Personalized\Image\Original\Happy\Photo-24.jpeg
C:\Ruchira\Academic\.IMMEDIATE\FYP\FYP\Da

In [24]:
# Define parameters
input_shape = (28, 28, 1)
num_classes = 5
num_episodes = len(dataset)
learning_rate = 0.001

# Create environment and model
environment = ImageEnvironment(dataset)
model = create_model(input_shape, num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [25]:
dataset[120]

(array([[0.25098039, 0.18039216, 0.16862745, 0.14509804, 0.7254902 ,
         0.7254902 , 0.6745098 , 0.47843137, 0.31764706, 0.16470588,
         0.03921569, 0.        , 0.        , 0.09411765, 0.        ,
         0.        , 0.60392157, 0.43137255, 0.        , 0.        ,
         0.        , 0.25882353, 0.00392157, 0.00784314, 0.21568627,
         0.10196078, 0.27058824, 0.56862745],
        [0.18823529, 0.16862745, 0.14117647, 0.11372549, 0.7372549 ,
         0.72941176, 0.66666667, 0.49019608, 0.25882353, 0.15294118,
         0.        , 0.01568627, 0.05490196, 0.00784314, 0.        ,
         0.01568627, 0.59215686, 0.42352941, 0.10980392, 0.02352941,
         0.        , 0.00392157, 0.01568627, 0.01176471, 0.21960784,
         0.17254902, 0.23921569, 0.54509804],
        [0.18039216, 0.19607843, 0.21960784, 0.3254902 , 0.74117647,
         0.7372549 , 0.63921569, 0.50588235, 0.3254902 , 0.1372549 ,
         0.02745098, 0.01176471, 0.00392157, 0.00392157, 0.35686275,
         0.

In [26]:
train_model(model, environment, optimizer, num_episodes)

Episode: 1, Loss: 1.0, Reward: -1
Episode: 2, Loss: 1.0, Reward: -1
Episode: 3, Loss: 1.0, Reward: -1
Episode: 4, Loss: 1.0, Reward: -1
Episode: 5, Loss: 1.0, Reward: 1
Episode: 6, Loss: 1.0, Reward: 1
Episode: 7, Loss: 1.0, Reward: -1
Episode: 8, Loss: 1.0, Reward: 1
Episode: 9, Loss: 1.0, Reward: 1
Episode: 10, Loss: 1.0, Reward: -1
Episode: 11, Loss: 1.0, Reward: -1
Episode: 12, Loss: 1.0, Reward: -1
Episode: 13, Loss: 1.0, Reward: -1
Episode: 14, Loss: 1.0, Reward: -1
Episode: 15, Loss: 1.0, Reward: -1
Episode: 16, Loss: 1.0, Reward: -1
Episode: 17, Loss: 1.0, Reward: -1
Episode: 18, Loss: 1.0, Reward: -1
Episode: 19, Loss: 1.0, Reward: -1
Episode: 20, Loss: 1.0, Reward: -1
Episode: 21, Loss: 1.0, Reward: -1
Episode: 22, Loss: 1.0, Reward: -1
Episode: 23, Loss: 1.0, Reward: -1
Episode: 24, Loss: 1.0, Reward: -1
Episode: 25, Loss: 1.0, Reward: -1
Episode: 26, Loss: 1.0, Reward: -1
Episode: 27, Loss: 1.0, Reward: -1
Episode: 28, Loss: 1.0, Reward: -1
Episode: 29, Loss: 1.0, Reward: -